<span style="color:blue; font-weight:bold; font-size:24px;">IMDB DATASET - CAPSTONE PROJECT PHASE 1</span>

<span style="color:brown; font-weight:bold; font-size:20px;">By SATYA JYOTHI</span>

<span style="color:brown; font-weight:bold; font-size:16px;">Batch: DS0622, DATATRAINED</span>

**1) Webscrape the provided URL  - IMDB dataset:**

https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1

**2) Store the entire data in two different CSV files as per the given fields:**

The first CSV file data contains :

Sno, Movie Name, Director Name, Duration, year, ratings, Metascore

Bifurcate the Director field into subfields as per the number of directors of the movie belongs to such as Director1, director2

The second CSV file contains the following:

Movie Name, stars, votes, Genre, Gross collection, popularity, Certification

Bifurcate the stars field into 4 subfields as per the number of stars worked in the movie such as star1, star2, star3, star4

Bifurcate the genre into 3 subfields as per the number of genres the movie belongs to such as :

Genre1, genre2, genre3

In [1]:
# calling the required webscrapping libraries
import pandas as pd #Importing pandas module
import selenium  #Importing the selenium module
from selenium import webdriver #Importing the webdriver module from selenium
from selenium.common.exceptions import NoSuchElementException #Importing specific exception classes from selenium
from selenium.webdriver.common.by import By #Importing the By class from selenium
import time #Importing the time module
import re #Importing the regular expressions module
from selenium.webdriver.support.ui import WebDriverWait # Importing the WebDriverWait class from selenium
import warnings # Import warnings module
warnings.filterwarnings("ignore") # Filtering warnings

## Creating first CSV file by webscrapping with selenium

### Sno, Movie Name, Director Name, Duration, year, ratings, Metascore

The first CSV file data contains :

Sno, Movie Name, Director Name, Duration, year, ratings, Metascore

Bifurcate the Director field into subfields as per the number of directors of the movie belongs to such as Director1, director2

In [2]:
# Connect to the driver
driver=webdriver.Chrome(r"chromedriver.exe")

# IMDb URL
url = 'https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1'

# Opening web page on automated chrome web browser
driver.get(url)
time.sleep(2)  # 2 seconds sleep to load the page

# maximize the window's size
driver.maximize_window()

#delete the cookies  
driver.delete_all_cookies()

# scrapping details: Sno, Movie Name, Director Name, Duration, year, ratings, Metascore

# I can extract all SNo., Movie Name, Duration, Year, Ratings details for all movies on a page without iterating for each movie container
# because all these variables have the data available for each movie however I can't do this for 'Metascore' as it is not available
# for some movies, hence for simplicity of the code, I iterated for each movie container to scrap all these details and replaced with blank
# whenever 'Metascore' is not present for a particular movie. However, iterating for each movie on a page took significant amount of time

# Create an empty dataframe df1 to store the results for 'SNo.','Movie Name','Duration (min)','Year','Ratings','Metascore'
df1 = pd.DataFrame(columns=['SNo.','Movie Name','Duration (min)','Year','Ratings','Metascore'])

# I can't extract 'directors' details for all movies on a page without iterating for each movie container
# Because each movie has different number of 'directors', hence iteration is required for each movie container

# Create an empty DataFrame df2 to store director names
df2 = pd.DataFrame()

start=0
end=35 # There are total 35 pages in IMDB website for web-scrapping all the data
mx_dir=0 # Variable for getting max number of directors

for page in range(start,end): # for loop for scrapping all pages of IMDB website
        
    # Find all movie containers on the page
    movie_containers = driver.find_elements(By.XPATH, '//div[@class="lister-item mode-advanced"]')
    
    # -------------------------------------------------------------------------------------------------------------------
    
    # Iterate through each movie container
    for container in movie_containers:
        
        # Find the maximum number of directors among all movies
        max_directors = max(len(container.find_elements(By.XPATH, './/p[contains(text(),"Director")]/a[(following-sibling::span[@class="ghost"])]')) for container in movie_containers)
        
        if max_directors > mx_dir:
            mx_dir = max_directors
        
        # Create column names for directors equivalent to maximum number of directors on a page as it is unknown upfront
        column_names = [f"Director {i+1}" for i in range(mx_dir)]
        
        # ******************************************************************************************************************

        # scrapping SNo
        try:
            SNo_element=container.find_element(By.XPATH,'.//h3[@class="lister-item-header"]//span[@class="lister-item-index unbold text-primary"]') 
            SNos = SNo_element.text.strip(" ")
            SNo = SNos.replace(",", "") # Removing commas
        except NoSuchElementException:
            SNo = ''  # Assign blank if SNo is not present
            
        # ******************************************************************************************************************
            
        # Get the movie title
        try:
            title_element = container.find_element(By.XPATH, './/h3[@class="lister-item-header"]/a')
            movie_title = title_element.text.strip(" ")
        except NoSuchElementException:
            movie_title = ''  # Assign blank if movie name is not present
            
        # ******************************************************************************************************************
        
        # scrapping Duration
        try:
            Duration_element=container.find_element(By.XPATH,'.//div[@class="lister-item-content"]/p[1]/span[3]') 
            Duration0=Duration_element.text.strip(" ")
            # Extract numerical numbers using regular expression. This is done to see only numbers which helps on queries
            numbers = re.findall(r'\d+', Duration0)
            # Join the numbers into a single string
            Duration = ''.join(numbers)
        except NoSuchElementException:
            Duration= ''  # Assign blank if Duration is not present
            
        # ******************************************************************************************************************

        # scrapping Year
        try:
            Year_element=container.find_element(By.XPATH,'.//div[@class="lister-item-content"]/h3/span[2]') 
            Year0=Year_element.text.strip(" ")
            # Extract numerical numbers using regular expression. This is done to see only numbers which helps on queries
            numbers = re.findall(r'\d+', Year0)
            # Join the numbers into a single string
            Year = ''.join(numbers)
        except NoSuchElementException:
            Year= ''  # Assign blank if Year is not present
            
        # ******************************************************************************************************************
    
        # scrapping Ratings
        try:
            Ratings_element=container.find_element(By.XPATH,'.//div[@class="lister-item-content"]/div/div[1]/strong') 
            Ratings=Ratings_element.text.strip(" ") 
        except NoSuchElementException:
            Ratings= ''  # Assign blank if Rating is not present
            
        # ******************************************************************************************************************
    
        # Find Metascore element
        try:
            metascore_element = container.find_element(By.XPATH, './/div[@class="lister-item-content"]/div/div[3]/span')
            metascore = metascore_element.text.strip(" ")
        except NoSuchElementException:
            metascore = ''  # Assign blank if Metascore is not present   
            
        # ******************************************************************************************************************
                
        # Find director elements
        director_elements = container.find_elements(By.XPATH, './/p[contains(text(),"Director")]/a[(following-sibling::span[@class="ghost"])]')
        directors = [director.text.strip() for director in director_elements]

        # Create a dictionary for the movie's directors
        try:
            movie_directors = {}
            for i in range(mx_dir):
                if i < len(directors):
                    movie_directors[column_names[i]] = directors[i]
                else:
                    movie_directors[column_names[i]] = ''
        except NoSuchElementException:
            movie_directors["Director 1"] = ''  # Assign blank if Director is not present
        
        # ******************************************************************************************************************
            
        # Add details to the dataframe df1
        df1 = df1.append({'SNo.':SNo,'Movie Name':movie_title,'Duration (min)':Duration,'Year':Year,'Ratings':Ratings,'Metascore':metascore}, ignore_index=True)
        
        # Append the movie's directors to the DataFrame df2
        df2 = df2.append(movie_directors, ignore_index=True)
        
        # ******************************************************************************************************************
               
    # Finding and Clicking on next button to go to next page
    if (page>=34):
        break
    else:
        next_button=driver.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/div[3]/div[1]/div/div[1]/div[2]/a[@class='lister-page-next next-page']")
        next_button.click()
        time.sleep(1)  # wait for 1 second to go to next line of code
    
    # -------------------------------------------------------------------------------------------------------------------
    
driver.close()  # closing the driver
    
# Concatenate the dataframes df1 and df2 on axis=1
df3 = pd.concat([df1, df2], axis=1)

# Replace NaN values in Director columns with blank because number of directors is not same for all movies
df4 = df3.fillna('')

#Displaying the dataframe
pd.set_option('display.max_rows', 20)
display(df4)

# Save the DataFrame 'df4' to a CSV file
df4.to_csv('IMDB_1.csv', index=False)

,SNo.,Movie Name,Duration (min),Year,Ratings,Metascore,Director 1,Director 2,Director 3,Director 4,Director 5,Director 6
0,1.,The Dark Knight,152,2008,9.0,84,Christopher Nolan,,,,,
1,2.,The Lord of the Rings: The Return of the King,201,2003,9.0,94,Peter Jackson,,,,,
2,3.,Inception,148,2010,8.8,74,Christopher Nolan,,,,,
3,4.,The Lord of the Rings: The Fellowship of the Ring,178,2001,8.8,92,Peter Jackson,,,,,
4,5.,The Lord of the Rings: The Two Towers,179,2002,8.8,87,Peter Jackson,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
1745,1746.,Radhe,109,2021,1.9,,Prabhu Deva,,,,,
1746,1747.,Race 3,160,2018,1.9,,Remo D'Souza,,,,,
1747,1748.,Angels Apocalypse,,2015,1.7,,Sean Cain,Enzo Zelocchi,,,,
1748,1749.,The Cost of Deception,,2021,1.5,,Keith English,,,,,


## Creating second CSV file by webscrapping with selenium

### Movie Name, stars, votes, Genre, Gross collection, popularity, Certification

The second CSV file contains the following:

Movie Name, stars, votes, Genre, Gross collection, popularity, Certification

Bifurcate the stars field into 4 subfields as per the number of stars worked in the movie such as star1, star2, star3, star4

Bifurcate the genre into 3 subfields as per the number of genres the movie belongs to such as :

Genre1, genre2, genre3

In [3]:
# Connect to the driver
driver=webdriver.Chrome(r"chromedriver.exe")

# Opening web page on automated chrome web browser
driver.get("https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1")
time.sleep(2)  # 2 seconds sleep

# maximize the window's size
driver.maximize_window()

#delete the cookies  
driver.delete_all_cookies()

# scrapping details: Movie Name, stars, votes, Genre, Gross collection, popularity, Certification

# I can extract all Movie_Name, Votes, Certification, Genre details for all movies on a page without iterating for each movie container
# because all these variables have the data available for all movies. I can do this by iterating for each movie container however,
# based on previous code experience which used to extract data for first csv file; took significant amount of time when iterating for each movie,
# hence wherever possible, I tried to scrap the data without iterating for each movie in this code and wherever it is required
# to iterate for each movie container; I used iterative approach. For example, I used iterative approach for scrapping 'stars' 
# and 'Gross Collection' details

# Creating empty lists.
Movie_Name=[]
Votes=[]
Certification=[]
# Eventhough 'Genre' has multiple elements like 'Action', 'Comedy', 'Drama' etc..but all together coming in a single xpath operation
# Initially I tried to extact all 'Genre' elements using single xpath operation and then segregated them at the end
Genre=[]  

# I can't extract all 'Gross Collection' details for all movies on a page without iterating for each movie container
# Because some of the movies don't have 'Gross Collection' details hence iteration is required for each movie container

# Create an empty dataframe df1 to store the 'Gross Collection'
df1 = pd.DataFrame(columns=['Gross (USD millions)'])

# I can't extract 'stars' details for all movies on a page without iterating for each movie container
# Because each movie has different number of stars, hence iteration is required for each movie container

# Create an empty DataFrame df2 to store 'stars' names
df2 = pd.DataFrame()

max_stars=0 # Variable for getting max number of stars 

start=0
end=35 # Max no. of pages in IMDB website

for page in range(start,end): # for loop for scrapping all pages of IMDB
    
    # -------------------------------------------------------------------------------------------------------------------     
    # scrapping Movie Name
    movie_name_info=driver.find_elements(By.XPATH,'//h3[@class="lister-item-header"]/a') 
    try:
        for i in movie_name_info:
            Movie_Name.append(i.text.strip(" ")) 
    except NoSuchElementException:
        Movie_Name.append("_")
    # -------------------------------------------------------------------------------------------------------------------        
    # scrapping Votes
    votes_elements=driver.find_elements(By.XPATH,'//div[@class="lister-item-content"]/p[4]/span[2]') 
    try:
        for i in votes_elements:
            Vote=i.text.strip(" ") 
            Votes.append(Vote.replace(",", "")) # Removing commas for easy queries
    except NoSuchElementException:
        Votes.append("_")
    # -------------------------------------------------------------------------------------------------------------------        
    # scrapping Certification
    Certification_elements=driver.find_elements(By.XPATH,'//div[@class="lister-item-content"]/p[1]/span[1]') 
    try:
        for i in Certification_elements:
            Certification.append(i.text.strip(" ")) 
    except NoSuchElementException:
        Certification.append("_")     
    # -------------------------------------------------------------------------------------------------------------------        
    # scrapping Genre
    Genre_elements=driver.find_elements(By.XPATH,'//div[@class="lister-item-content"]//span[@class="genre"]') 
    try:
        for i in Genre_elements:
            Genre.append(i.text.strip(" ")) 
    except NoSuchElementException:
        Genre.append("_")   
    # -------------------------------------------------------------------------------------------------------------------
    # Find all movie containers on the page
    movie_containers = driver.find_elements(By.XPATH, '//div[@class="lister-item mode-advanced"]')
    
    ######################################################################################################################    
    # Iterate through each movie container
    for container in movie_containers:
        
        # Find Gross Collection
        try:
            gross_collection_element = container.find_element(By.XPATH, './/div[@class="lister-item-content"]/p[4]/span[5]')
            gross_collections = gross_collection_element.text.strip(" ")
            gross_collection = gross_collections.replace("$", "").replace("M", "") # Remove $ and M for easy queries
        except NoSuchElementException:
            gross_collection = ''  # Assign blank if Gross_collection is not present  
        # -------------------------------------------------------------------------------------------------------------------
        # Find the maximum number of stars among all movies
        stars_count = max(len(container.find_elements(By.XPATH, './/p[contains(text(),"Director")]/a[not(following-sibling::span[@class="ghost"])]')) for container in movie_containers)
        
        if stars_count > max_stars:
            max_stars = stars_count
        
        # Create column names for stars
        column_names_star = [f"Star {i+1}" for i in range(max_stars)]
        
        # Find star elements
        star_elements = container.find_elements(By.XPATH, './/p[contains(text(),"Director")]/a[not(following-sibling::span[@class="ghost"])]')
        stars = [star.text.strip() for star in star_elements]

        # Create a dictionary for the movie's stars
        try:
            movie_stars = {}
            for i in range(max_stars):
                if i < len(stars):
                    movie_stars[column_names_star[i]] = stars[i]
                else:
                    movie_stars[column_names_star[i]] = ''
        except NoSuchElementException:
            movie_stars["Star 1"] = ''  # Assign blank if Star is not present
            
        ###################################################################################################################### 
        # Add the gross details to the dataframe 'df1'
        df1 = df1.append({'Gross (USD millions)':gross_collection}, ignore_index=True)
               
        # Append the movie's stars to the DataFrame 'df2'
        df2 = df2.append(movie_stars, ignore_index=True)       
        # -------------------------------------------------------------------------------------------------------------------
    
    # Finding and Clicking on next button to go to next page
    if (page>=34):
        break
    else:
        next_button=driver.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/div[3]/div[1]/div/div[1]/div[2]/a[@class='lister-page-next next-page']")
        next_button.click()
        time.sleep(1)  # wait for 1 second to go to next line of code
        
driver.close()  # closing the driver

# creating a data frame df0 with initial information scrapped ('Movie Name', 'Votes', Certification','Genre')
df0=pd.DataFrame({'Movie Name':Movie_Name,'Votes':Votes,'Certification':Certification,'Genre':Genre})
        
# Concatenate the dataframes df0, df1, df2 on axis=1
df3 = pd.concat([df0, df1, df2], axis=1)

# -------------------------------------------------------------------------------------------------------------------
# Split the values in the "Genre" column
max_cols = (df3['Genre'].str.count(',') + 1).max()
col_names = [f'Genre{i+1}' for i in range(max_cols)]
df3[col_names] = df3['Genre'].str.split(',', expand=True)

# Replace NaN values with blank in "Stars" and "Genre" columns because number of 'stars' and 'Genre values' are not same for all movies
df4 = df3.fillna('')

# Drop the original "Genre" column
df4.drop('Genre', axis=1, inplace=True)

In [4]:
# -------------------------------------------------------------------------------------------------------------------------

# 'popularity' details are not available on IMDB webpage directly. I need to click on each movie to go to another url to scrap poplularity details for each movie

# scrapping popularity details of all movies on IMDB

# Connect to the driver
driver=webdriver.Chrome(r"chromedriver.exe")

# Opening web page on automated chrome web browser
driver.get("https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1")
time.sleep(2)  # 2 seconds sleep

# maximize the window's size
driver.maximize_window()

#delete the cookies  
driver.delete_all_cookies()

# Creating empty lists for url links for all movies and movie popularity details
Movie_urls=[]
popularity=[]

# -------------------------------------------------------------------------------------------------------------------

start=0
end=35  # Max no. of pages in IMDB website

for page in range(start,end): # for loop for scrapping all pages of IMDB  to get urls of all movies
    
    # getting all urls of movies displayed on IMDB page
    url_movie=driver.find_elements(By.XPATH,'//h3[@class="lister-item-header"]/a')
    for i in url_movie:  # for loop getting the POPULARITY data from urls 
        Movie_urls.append(i.get_attribute("href"))  # getting urls through href attribute    
    # Finding and Clicking on next button to go to next page
    if (page>=34):
        break
    else:
        next_button=driver.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/div[3]/div[1]/div/div[1]/div[2]/a[@class='lister-page-next next-page']")
        next_button.click()
        time.sleep(1)  # wait for 1 second to go to next line of code
        
# -------------------------------------------------------------------------------------------------------------------

for i in Movie_urls: # loop for every movie url in the list to get popularity details
    driver.get(i)  # Opening each movie url 
    driver.implicitly_wait(3) # wait the driver for 3 seconds
    
    #getting Populariy details of a movie
    try: 
        popularity_elements=driver.find_element(By.XPATH,'/html/body/div[2]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[3]/a/span/div/div[2]/div[@class="sc-5f7fb5b4-1 bhuIgW"]')
        popularitys = popularity_elements.text.strip(" ")
        popularity.append(popularitys.replace(",", ""))
    except NoSuchElementException:   # exception handling for Repository popularity
        popularity.append('')       # replace with "_" for missing details

# -------------------------------------------------------------------------------------------------------------------
        
driver.close()  # closing the driver
                                                
# creating a data frame df5 with all the information scrapped for popularity
df5=pd.DataFrame({'Popularity':popularity})

# Concatenate the dataframes df4 and df5 on axis=1
df6 = pd.concat([df4, df5], axis=1)

#Displaying the updated dataframe
pd.set_option('display.max_rows', 20)
display(df6)

# Save the DataFrame 'df6' to a CSV file
df6.to_csv('IMDB_2.csv', index=False)

,Movie Name,Votes,Certification,Gross (USD millions),Star 1,Star 2,Star 3,Star 4,Genre1,Genre2,Genre3,Popularity
0,The Dark Knight,2718923,PG-13,534.86,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,Action,Crime,Drama,117
1,The Lord of the Rings: The Return of the King,1887507,PG-13,377.85,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,Action,Adventure,Drama,407
2,Inception,2413616,PG-13,292.58,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,Action,Adventure,Sci-Fi,103
3,The Lord of the Rings: The Fellowship of the Ring,1916139,PG-13,315.54,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean,Action,Adventure,Drama,184
4,The Lord of the Rings: The Two Towers,1703659,PG-13,342.55,Elijah Wood,Ian McKellen,Viggo Mortensen,Orlando Bloom,Action,Adventure,Drama,788
...,...,...,...,...,...,...,...,...,...,...,...,...
1745,Radhe,178808,TV-MA,,Salman Khan,Disha Patani,Randeep Hooda,Jackie Shroff,Action,Crime,Thriller,
1746,Race 3,47589,Not Rated,1.69,Anil Kapoor,Salman Khan,Bobby Deol,Jacqueline Fernandez,Action,Crime,Thriller,
1747,Angels Apocalypse,42911,85 min,,Enzo Zelocchi,Jana Rochelle,Ryan C.F. Buckley,William Kirkham,Action,Fantasy,Sci-Fi,
1748,The Cost of Deception,39487,125 min,,Vivianne Bánovits,András Mózes,Barna Bokor,Gabriella Gubás,Action,Crime,Drama,
